# Setup

In [1]:
import pandas as pd
import numpy as np
import random
import traceback
from os import path
DATA_DIR = path.join('..', 'data')
NP_TYPE = np.double
np.seterr(all='raise')

{'divide': 'warn', 'invalid': 'warn', 'over': 'warn', 'under': 'ignore'}

# Initial state

In [2]:
def check_state(S, I, R, N):
    assert (N == S + I + R).all() # Checks both value and shapes
    assert (S >= 0).all()
    assert (R >= 0).all()
    assert (I >= 0).all()
    assert np.isclose(N.sum(), POP_SIZE)

In [3]:
boroughs = pd.read_csv(path.join(DATA_DIR, 'borough_pop.csv'))
stations = pd.read_csv(path.join(DATA_DIR, 'station_borough.csv'))
POP_SIZE = boroughs['Population'].sum()
POP_SIZE

7947055

In [4]:
borough_count = stations['Local authority'].value_counts().to_frame()
borough_count.columns = ['Station count']
boroughs_pop_count = boroughs.merge(borough_count, left_on='Local authority',
                                    right_index=True, validate='one_to_one')
boroughs_pop_count['Station population'] = boroughs_pop_count['Population'] / boroughs_pop_count['Station count']
boroughs_pop_count.head()

,Local authority,Population,Station count,Station population
0,Havering,234127,6,39021.166667
1,Wandsworth,299347,8,37418.375000
2,Harrow,233495,12,19457.916667
3,Croydon,352763,7,50394.714286
4,Lewisham,270418,8,33802.250000


In [5]:
stations_pop = stations.merge(boroughs_pop_count).sort_values('Station')
INITIAL_N = stations_pop['Station population'].values

# Transition matrices

## Load data

In [6]:
DAY_LOOKUP = {
    'Mon': 0,
    'Tue': 1,
    'Wed': 2,
    'Thu': 3,
    'Fri': 4,
    'Sat': 5,
    'Sun': 6,
}

In [7]:
move_data = pd.read_csv(path.join(DATA_DIR, 'journey_count.csv'))
move_data['Day'].replace(DAY_LOOKUP, inplace=True)
move_data.columns = ['Start', 'End', 'Day', 'Hour', 'Journeys']
move_data.loc[move_data['Hour'] > 23, 'Day'] += 1
move_data.loc[move_data['Hour'] > 23, 'Hour'] -= 24
move_data.head()

,Start,End,Day,Hour,Journeys
0,Acton Central,Acton Central,5,18,1
1,Acton Central,Acton Central,6,7,1
2,Acton Central,Acton Central,3,13,1
3,Acton Central,Acton Central,2,17,1
4,Acton Central,Baker Street,0,7,1


In [28]:
STATION_LOOKUP = {
    name: i for i, name in enumerate(move_data['Start'].unique())
}
ID_LOOKUP = {
    i: name for name, i in STATION_LOOKUP.items()
}

## Create matrices

In [9]:
def calc_hour(day, hour):
    return day * 24 + hour

max_day = move_data['Day'].max()
max_day_max_hour = move_data[move_data['Day'] == max_day]['Hour'].max()
hourly_F = [
    np.zeros((len(STATION_LOOKUP), len(STATION_LOOKUP)))
    for _ in range(calc_hour(max_day, max_day_max_hour) + 1)
]

STATION_POP = {
    row['Station']: row['Station population'] for _, row in stations_pop.iterrows()
}

for row in move_data.itertuples():
    start = STATION_LOOKUP[row.Start]
    end = STATION_LOOKUP[row.End]
    if start != end:
        hourly_F[calc_hour(row.Day, row.Hour)][start][end] = 20*row.Journeys

In [10]:
def check_F(F):
    assert F.shape == (len(INITIAL_N), len(INITIAL_N))
    assert (F.sum(axis=1) <= 1).all()
# for F in hourly_F:
#     check_F(F)

# Constants

In [11]:
BETA = NP_TYPE(0.5 / 24)
GAMMA = NP_TYPE((1/3) / 24)
assert np.isclose(BETA / GAMMA, 1.5)

# Main

In [22]:
def update_state(F, Fdash, S, I, R, N):
    S_I_interaction = BETA * S * I * 1/N
    Snew = -S_I_interaction + F.T.dot(S) - Fdash * S + S
    Inew = S_I_interaction + F.T.dot(I) - Fdash * I + (1-GAMMA) * I
    Rnew = GAMMA * I + F.T.dot(R) - Fdash * R + R
    Nnew = Snew + Inew + Rnew
    return (Snew, Inew, Rnew, Nnew)

def run_simulation(state, start_time=0, timesteps=None):
    old_err = np.seterr(under='ignore')
    t = 0
    Stotals = Itotals = Rtotals = []
    Itotal = sum(state[1])
    while Itotal > 0.5 and (timesteps == None or t < timesteps):
#         if t % 1000 == 0:
#             print('{}: {} infected'.format(t, Itotal))
#         Stotals.append(S.sum())
#         Itotals.append(Itotal)
#         Rtotals.append(R.sum())
#         print(state)
        F = hourly_F[t % len(hourly_F)] / state[3].reshape((state[3].shape[0], 1))
        mask = F.sum(axis=1) > 1
        if mask.any():
            F[mask] = F[mask,] / (F[mask].sum(axis=1).reshape(F[mask].shape[0], 1) + 1e-10)
            print('Adjusting too high F: timestamp {}, stations {}'\
                  .format(t, ', '.join(ID_LOOKUP[i] for i, cond in enumerate(mask)
                              if cond)))
        try:
            check_F(F)
        except AssertionError:
            print('err')
            return t, state[0]
        Fdash = F.sum(axis=1)
        new_state = update_state(F, Fdash, *state)
#         check_state(*new_state)
        state = new_state
        t += 1
#         assert len(states) == t
        Itotal = sum(state[1])
    np.seterr(**old_err)
    return state

In [24]:
# def generate_state():
#     N = INITIAL_N
#     I = np.zeros(len(N))
#     R = np.zeros(len(N))
#     I[50] = 1000
#     S = N - I
#     return (S, I, R, N)

# %timeit run_simulation(generate_state(), timesteps=1000)

In [30]:
START_TIMES = [24*i for i in range(7)]
INITIAL_INFECTEDS = (1, 10, 10000)
HEADER = ('Init_station', 'Init_time', 'Init_count', 'Count_type')
HEADER_PRINT = 'Starting at: station {}, time {}, count {}'
import csv
# with open('results.csv', 'w', newline='') as outfile:
#     writer = csv.writer(outfile)
#     writer.writerow(HEADER)
for station_index in range(len(INITIAL_N)):
    print('Station {} of {}'.format(station_index, len(INITIAL_N)))
    for I_count in INITIAL_INFECTEDS:
        if INITIAL_N[station_index] < I_count:
            print('Too small population to start with {}'.format(I_count))
            break
        for t in START_TIMES:
            N = INITIAL_N
            I = np.zeros(len(N))
            R = np.zeros(len(N))
            I[station_index] = I_count
            S = N - I
            state = (S, I, R, N)
            try:
                check_state(*state)
                result = run_simulation(state=state, start_time=t)
            except Exception as err:
                print('ERROR: start time is {}, infecteds is {}'.format(t, I_count))
                raise
#                 else:
#                     for i, name in enumerate(('S', 'I', 'R')):
#                         outrow = [station_index, t, I_count, name]
#                         outrow.extend(result[i])
#                         writer.writerow(outrow)

Station 0 of 395
Adjusting too high F: timestamp 18, stations Cannon Street, Liverpool Street
Adjusting too high F: timestamp 19, stations Cannon Street, Liverpool Street
Adjusting too high F: timestamp 20, stations Bank, Cannon Street, Liverpool Street
Adjusting too high F: timestamp 21, stations Bank, Liverpool Street
Adjusting too high F: timestamp 22, stations Bank, Liverpool Street
Adjusting too high F: timestamp 23, stations Bank, Liverpool Street
Adjusting too high F: timestamp 29, stations Bank
Adjusting too high F: timestamp 30, stations Cannon Street, Liverpool Street
Adjusting too high F: timestamp 31, stations Cannon Street
Adjusting too high F: timestamp 32, stations Cannon Street
Adjusting too high F: timestamp 42, stations Cannon Street, Liverpool Street
Adjusting too high F: timestamp 43, stations Cannon Street, Liverpool Street
Adjusting too high F: timestamp 44, stations Bank, Cannon Street, Liverpool Street
Adjusting too high F: timestamp 45, stations Bank, Liverpool

FloatingPointError: divide by zero encountered in true_divide